# Real-Time_People_Counting


In [1]:
import os 
import re
import glob
import spark
import pandas as pd
import functools
import requests
import json
import time
import datetime
import numpy as np
import scipy as spy
from collections import defaultdict
from datetime import timedelta
from copy import deepcopy
from multiprocessing import Pool
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from tqdm import tqdm

pj = os.path.join
datapath = '/Users/kanghuang/Documents/work/WeFi/data/japan'

### Data Loader

In [2]:
def get_ap_log(bids, days=60):
    df_list = []
    c = 0
    cur_date = pd.to_datetime('now')
    start_date = cur_date - timedelta(days=days)
    start_date = start_date.replace(minute=0, hour=0, second=0)
    end_date = cur_date.replace(minute=0, hour=0, second=0)
    for path in glob.glob(pj(datapath, '*/*.parquet')):
        ap_log = pd.read_parquet(path)
        try:
            ap_log['localTimestamp'] = pd.to_datetime(ap_log['localTimestamp'])
            ap_log = ap_log[(ap_log['buildingId'].isin(bids)) 
#                            &(ap_log['localTimestamp'] > start_date) 
#                            &(ap_log['localTimestamp'] < end_date)
                           &(ap_log['ssid'] != "")]
            if len(ap_log) == 0:
                c += 1
            df_list.append(ap_log)
        except:
            print (ap_log.columns)
    df = pd.concat(df_list)
    return df


bids = [
    #27, #'80654332-e7df-11e8-b223-0626947414d8'
    22, # '4551855b-7eee-4fb4-8f43-5756c8dde881' 
]

all_ap_df = get_ap_log(set(bids))

# 'abnormal'
# ae301ff2d6f111ea8bde16245ec60c24, 5ad0f61cd55f11ea8bde16245ec60c24, acf38bb0d6f111ea95420242ac130004,
# 83118d7ad62811eaaaea0242ac130004, 2ef054dad49611ea8bde16245ec60c24, 584d7852d55f11ea8bde16245ec60c24,
# 2f10947ad49611ea87f90242ac130004, af5b284ad6f111ea835f0242ac130004, 83ca466cd62811ea8bde16245ec60c24,
# 81ed97d6d62811ea87ca0242ac130004, f716d86ed69711ea8bde16245ec60c24, 83cc7572d62811ea8bde16245ec60c24,
# abce3212d6f111ea8bde16245ec60c24, 02acdcb4d3cd11ea9d7f0242ac130004, 2fe33f38d49611ea8bde16245ec60c24,
# 2ca1379ed49611eab69c0242ac130004, fcd7b26ed69c11ea8bde16245ec60c24, 

Index(['clientId', 'campusId', 'buildingId', 'floorId', 'floorNumber', 'apMac',
       'apManufacturer', 'deviceMac', 'manufacturerId', 'timeLocal',
       'timezone', 'date', 'isApple', 'knownOUI', 'rssi', 'ssid'],
      dtype='object')
Index(['clientId', 'campusId', 'buildingId', 'floorId', 'floorNumber', 'apMac',
       'apManufacturer', 'deviceMac', 'manufacturerId', 'timeLocal',
       'timezone', 'date', 'isApple', 'knownOUI', 'rssi', 'ssid'],
      dtype='object')
Index(['clientId', 'campusId', 'buildingId', 'floorId', 'floorNumber', 'apMac',
       'apManufacturer', 'deviceMac', 'manufacturerId', 'timeLocal',
       'timezone', 'date', 'isApple', 'knownOUI', 'rssi', 'ssid'],
      dtype='object')
Index(['clientId', 'campusId', 'buildingId', 'floorId', 'floorNumber', 'apMac',
       'apManufacturer', 'deviceMac', 'manufacturerId', 'timeLocal',
       'timezone', 'date', 'isApple', 'knownOUI', 'rssi', 'ssid'],
      dtype='object')
578


In [3]:
bid = 22
valid_ap_df = all_ap_df[(all_ap_df['buildingId'] == bid) & (all_ap_df['floorNumber'] != 999)]
valid_ap_df = valid_ap_df.rename(columns={'deviceMac': 'uuid', 'floorNumber': 'floor_number'})

In [4]:
valid_ap_df['date'] = pd.to_datetime(valid_ap_df['date'])
valid_ap_df['type'] = 'device'

In [5]:
def filter_outlier(row):
    # overnight
    timestamp = pd.to_datetime(row.name)
    earlyTimestamp = timestamp.replace(minute=0, hour=6, second=0)
    lateTimestamp = timestamp.replace(minute=0, hour=23, second=0)
    overnight = (row['localTimestamp'] < earlyTimestamp) | (row['localTimestamp'] > lateTimestamp)
    row =  row[~overnight]
#     row['date'] = pd.to_datetime(row['date'])
#     row['dow'] = row.date.apply(lambda x: x.weekday())
#     row = row[row.dow >= 5]
    return row

keycard_swipes_df = pd.read_csv('keycard_swipes_japan.csv')
keycard_swipes_df['date'] = keycard_swipes_df.apply(lambda row: row['KEYCARD_SWIPE_LOCAL'].split()[0], axis=1)
keycard_swipes_df = keycard_swipes_df.rename(columns={'KEYCARD_SWIPE_LOCAL': 'localTimestamp', 'CUSTOMER_UUID': 'uuid'})
keycard_swipes_df['localTimestamp'] = pd.to_datetime(keycard_swipes_df['localTimestamp'])
keycard_swipes_df['date'] = pd.to_datetime(keycard_swipes_df['date'])
keycard_swipes_df = keycard_swipes_df[keycard_swipes_df['date'].isin(valid_ap_df.date.unique())]
valid_keycard_df = deepcopy(keycard_swipes_df)
valid_keycard_df = valid_keycard_df[(valid_keycard_df.READER_NAME != 'Proxy Reader') & (valid_keycard_df.ACTIVITY == 'Access granted')]
valid_keycard_df = valid_keycard_df.groupby('date').apply(filter_outlier).reset_index(drop=True)
valid_keycard_df['floor_number'] = valid_keycard_df.READER_NAME.str.extract(r'(\d{2})\.\d+').fillna('') 
valid_keycard_df = valid_keycard_df[valid_keycard_df.floor_number != '']
valid_keycard_df['type'] = 'keycard'


### Preprocess function

In [7]:
def filter_stationary(row, keycard_flag):
    # overnight
    timestamp = pd.to_datetime(row.name)
    earlyTimestamp = timestamp.replace(minute=0, hour=6, second=0)
    lateTimestamp = timestamp.replace(minute=0, hour=23, second=0)
    if keycard_flag:
        overnight = (row['localTimestamp'] < row['min']) 
    else:
        overnight = row['localTimestamp'] < earlyTimestamp
    row =  row[overnight]
#     row['date'] = pd.to_datetime(row['date'])
#     row['dow'] = row.date.apply(lambda x: x.weekday())
#     row = row[row.dow >= 5]
    return row

def smallest_stationary_group(ap_log):
    ap_log = ap_log.groupby('uuid').apply(lambda row: row['date'].nunique())
    day_n = ap_log.max()
    ap_log = ap_log[ap_log == day_n]
    return ap_log.index.tolist()

def largest_stationary_group(ap_log):
#     day_n = len(ap_log.date.unique())
#     ap_log = ap_log.groupby('uuid').apply(lambda row: row['date'].nunique())
#     ap_log = ap_log[ap_log > 0]
    return ap_log.index.tolist()

def gen_stationary_group(valid_ap_df, group_func, keycard_swipes_df=None):
    #overlap, vocab = cooccurrence_model
    keycard_flag=False
    if keycard_swipes_df is not None:
        boundary_df = keycard_swipes_df.groupby('date')['localTimestamp'].agg(['min'])
        valid_ap_df = pd.merge(valid_ap_df, boundary_df, on='date')
        keycard_flag = True
        
    overnight_ap_log = valid_ap_df.groupby('date').apply(lambda df: filter_stationary(df, keycard_flag)).reset_index(drop=True)
    stationary_group = group_func(overnight_ap_log)
    return set(stationary_group)

def filter_low_rssi(valid_ap_df):
    rssi = -80
    rssi_cond = (valid_ap_df['rssi'] < 0) & (valid_ap_df['rssi'] > rssi)
    return valid_ap_df[rssi_cond]

def filter_weekend(df):
    df['date']  = pd.to_datetime(df['date'])
    df['dow'] = df.date.apply(lambda x: x.weekday())
    return df[df.dow < 5]

### Device to device graph

In [8]:
'''
This function calculate the cooccurrence count within a single timespan.
There are two groups to maintain the state, left group records device ids from previous time window,
right group records new incoming device ids.
There are three conditions to update the cooccurrence pairs:
1. left device is from left group, right device is from and only exists in right group without occur in left device's timespan 
2. left device is from right group, right device is from and only exists in left group.
3. both devices are from right group
'''
def count_cooccurrence(l, r, rr, cooccurrence, obsolete_index, devices):
    """
    l: left index of left group
    """
    left_freq = defaultdict(int)
    right_freq = defaultdict(int)
    for i in devices[l: r]:
        left_freq[i] += 1
    for i in devices[r: rr]:
        right_freq[i] += 1
        
    rd = right_freq.keys()
    ld = left_freq.keys()
    prev = -1
    obsolete_devices = set()
    for i in rd:
        if not i in cooccurrence:
            cooccurrence[i] = defaultdict(int)
    # condition 1
    for i in rd: 
        if (not i in ld):
            for k in range(l, r):
                j = devices[k]
                st = obsolete_index[k]
                if prev != st:
                    obsolete_devices = set(devices[st: k]) 
                    prev = st
                if not i in obsolete_devices: 
                    cooccurrence[j][i] += 1
    # condition 2               
    for i in rd: 
        for j in ld:
            if not j in rd: 
                cooccurrence[i][j] += right_freq[i]
    
    # condition 3
    for i in rd:
        for j in rd: 
            if i != j:
                cooccurrence[i][j] += right_freq[i]    
                
"""
This function slides the time window from left to right in steps of timestamps,
every time left index will try to shift one timestamp to right direction, and right index will find the 
right most index within the time window starts from left index.
It will also maintain every device left most index within time window
"""                
def create_context(timespan, df, cooccurrence):
    df = df.sort_values('localTimestamp')
    timestamps = df['localTimestamp'].tolist()
    devices = df['uuid'].tolist()
    l, r = 0, 0 
    obsolete_index = []
    while True:
        ll = l
        while ll < r:
            assert timestamps[ll] >= lt
            if timestamps[ll] != lt:
                break
            ll += 1
        if ll == len(timestamps):
            break
        l = ll
        lt = timestamps[l]
        rr = r
        while (rr < len(timestamps)) and (timestamps[rr] < lt + timedelta(minutes=timespan)):
            rr += 1
        if r == rr: continue
        obsolete_index.extend([l]*(rr-r))
        count_cooccurrence(l, r, rr, cooccurrence, obsolete_index, devices)
        r = rr
    assert len(obsolete_index) == len(devices)
        
def cooccurrence_update(metadata, ap_df, timespan, group_columns=['date', 'apMac', 'floor_number']):
    cooccurrence, freq = metadata
    for mac in ap_df['uuid'].tolist():
        freq[mac] += 1
    for _, df in ap_df.groupby(group_columns):
        create_context(timespan, df, cooccurrence)
    return cooccurrence, freq

        
def cooccurrence_transform(metadata, thresh=0.5, binary=True):
    cocur, freq = metadata
    indptr = [0]
    indices = []
    data = []
    vocab = {}
    for i in cocur:
        vocab.setdefault(i, len(vocab))
        for j in cocur[i]:
            index = vocab.setdefault(j, len(vocab))
            connectivity = cocur[i][j]*1.0 / freq[i]
            if binary:
                if connectivity > thresh:
                    connectivity = 1
                else:
                    connectivity = 0
            if connectivity > 0:
                data.append(connectivity)
                indices.append(index)
        indptr.append(len(indices))
    n = len(cocur)
    graph = csr_matrix((data, indices, indptr), shape=(n, n), dtype=float)
    return graph, vocab


In [9]:
def create_index():
    date_start = datetime.datetime(2020, 1, 6)
    date_end = datetime.datetime(2020, 3, 24)
    date_groups = []
    cur_date = date_start
    while cur_date <= date_end:
        date_groups.append((cur_date, min(date_end, cur_date+timedelta(days=6))))
        cur_date += timedelta(days=7)
    return date_groups

def prepare_data(k, date_groups):
    train_ap_df = valid_ap_df[(valid_ap_df.date >= date_groups[k][0]) & (valid_ap_df.date <= date_groups[k][1])]
    test_ap_df = valid_ap_df[(valid_ap_df.date >= date_groups[k+1][0]) & (valid_ap_df.date <= date_groups[k+1][1])]
    train_keycard_df = valid_keycard_df[(valid_keycard_df.date >= date_groups[k][0]) & (valid_keycard_df.date <= date_groups[k][1])]
    test_keycard_df = valid_keycard_df[(valid_keycard_df.date >= date_groups[k+1][0]) & (valid_keycard_df.date <= date_groups[k+1][1])]
    return train_ap_df, test_ap_df, train_keycard_df, test_keycard_df

def preprocess(ap_df, group_func, keycard_df=None):
    key_columns = ['localTimestamp', 'floor_number', 'uuid', 'apMac']
    print ('before preprocess %d' % len(ap_df))
    ap_df = filter_low_rssi(ap_df)
    ap_df = ap_df[~ap_df.duplicated(key_columns)]
    stationary_group = gen_stationary_group(ap_df, group_func, keycard_df)
    print ('after preprocess %d' % len(ap_df))
    return ap_df, stationary_group

In [10]:


def ssg(df, vocab, labels, static_group):
    candidate = defaultdict(int)
    uuids = [vocab[i] for i in df.uuid.unique() if i in vocab]
    new_device_n = (df.uuid.nunique() - len(uuids)) // 2
    for i, label in enumerate(labels):
        if i in uuids:
            candidate[label] += 1
    #print (len(uuids), len(labels))
    if len(candidate) == 0:
        return new_device_n
    c = 0
    for label in candidate:
        if (not label in static_group) and (candidate[label] <= 3):
            c += 1
    return c + new_device_n

def ssg_predict(train_keycard_df, test_keycard_df, test_ap_df, model, stationary_group): 
    test_df = filter_low_rssi(test_ap_df)
    #test_df = test_df[~test_df.isin(stationary_group)]
    estimate = test_df.groupby('date').apply(lambda df: ssg(df, model, stationary_group))
    keycard = test_keycard_df.groupby('date').apply(lambda rows: rows.uuid.nunique())
    delta = test_keycard_df.groupby('date').apply(lambda rows: rows[~rows.uuid.isin(train_keycard_df.uuid.unique())].uuid.nunique())
    device = test_df.groupby('date').apply(lambda rows: rows.uuid.nunique())
  
    dashboard = pd.concat([estimate, keycard, device, delta], axis=1).reset_index(drop=False)
    dashboard = dashboard[~dashboard[0].isna()]
    
    #truth = pd.read_csv('japan/japan_manual_counts.csv')
    return dashboard

def get_correlation(df):
    pearson1 = df.manual_count.corr(df.estimate)
    pearson2 = df.manual_count.corr(df.baseline)
    return pearson1, pearson2
 
def get_error_rate(df):
    err1 = np.sum((np.abs(df.estimate-df.manual_count)) / (df.manual_count+1)) / len(df)
    err2 = np.sum((np.abs(df.baseline-df.manual_count)) / (df.manual_count+1)) / len(df)
    return err1, err2

def get_mae(df):
    mae1 = np.sum(np.abs(df.estimate-df.manual_count)) / len(df)
    mae2 = np.sum(np.abs(df.baseline-df.manual_count)) / len(df)
    return mae1, mae2

def ssg_evaluate(ratio, truth, test_ap_df, model, stationary_ssg, stationary_ratio, common_area_aps=None):  
    graph, vocab = model
    test_df = filter_low_rssi(test_ap_df)
    #test_df = test_df[~test_df.isin(stationary_group)]
    truth['time_local'] = pd.to_datetime(truth['time_local'])
    truth = truth[(truth['time_local'] >= test_df.date.min()) & (truth['time_local'] <= test_df.date.max())].reset_index(drop=True)
    n_components, labels = connected_components(csgraph=graph, connection='strong', return_labels=True)
    
    static_group = set()
    stationary_ssg_ids = set([vocab[mac] for mac in stationary_ssg])
    for i, label in enumerate(labels):
        if i in stationary_ssg_ids:
            static_group.add(label)
    if not common_area_aps is None:
        truth = truth[truth['floor'].isin(common_area_aps)]
        
    for i, row in truth.iterrows():
        timestamp = row['time_local']
        cond = (test_df['localTimestamp'] > (timestamp - timedelta(minutes=5))) & (test_df['localTimestamp'] < (timestamp + timedelta(minutes=5)))
        cond &= (test_df['floor_number'] == row['floor'])
        if not common_area_aps is None:
            cond &= (test_df['apMac'].isin(common_area_aps[row['floor']]))
        ap_df = test_df[cond] 
        truth.loc[i, 'baseline'] = round(ap_df[~ap_df.uuid.isin(stationary_ratio)].uuid.nunique() * ratio)
        truth.loc[i, 'estimate'] = ssg(ap_df, vocab, labels, static_group)
        truth.loc[i, 'device'] =  ap_df.uuid.nunique() 
        
        #print (ap_df.uuid.nunique(), row['manual_count'], count, timestamp, row['floor'])
        
    truth = truth.sort_values(['floor', 'time_local']).reset_index(drop=True)
    cond = (truth['device'] == 0) & (truth['manual_count'] != 0)
    cond |= ((truth['device'] != 0) & (truth['manual_count'] == 0))
    cond |= (truth['device'] < truth['manual_count'])
    res = truth[~cond]
    err1, err2 = get_error_rate(res)
    mae1, mae2 = get_mae(res)
    pear1, pear2 = get_correlation(res)
    return res, err1, err2, mae1, mae2, pear1, pear2


In [11]:
def lr_fit(train_ap_df, train_keycard_df):
    train_keycard_df = filter_weekend(train_keycard_df)
    train_ap_df = filter_weekend(train_ap_df)
    train_df, stationary_group = preprocess(train_ap_df, largest_stationary_group)
    train_df = train_df[~train_df.uuid.isin(stationary_group)]
    pseudo_gold = train_keycard_df.groupby('date').apply(lambda row: row.uuid.nunique())
    train = train_df.groupby('date').apply(lambda row: row.uuid.nunique())
    y = pseudo_gold.to_numpy().astype('float64') 
    x = train.to_numpy().astype('float64')
    X = np.expand_dims(x, -1)
    a, _, _, _ = np.linalg.lstsq(X, y)
    return a[0], stationary_group

In [12]:
st = time.time()
truth = pd.read_csv('japan/japan_manual_counts.csv')
truth = truth[truth['building_id'] == 22]
date_groups = create_index()
metadata = {}, defaultdict(int)
stationary_global = set()
start = time.time()
metadata_versions = []
test_data = []
stationary_ssg_groups = []
stationary_ratio_groups = []
for k in range(len(date_groups)-1):
    print ('========iteration %d=========' % k)
    train_ap_df, test_ap_df, train_keycard_df, test_keycard_df = prepare_data(k, date_groups)
    train_ap_df, stationary_group = preprocess(train_ap_df, smallest_stationary_group, train_keycard_df)
    stationary_global = stationary_global.union(stationary_group)
    print ('stationary group size: %d' % len(stationary_global))
    metadata = cooccurrence_update(metadata, train_ap_df, 5)
    model = cooccurrence_transform(metadata)
    print ('graph running time: %ds' % (time.time()-st))
    #st = time.time()
    #ssg_predict(train_keycard_df, test_keycard_df, test_ap_df, model, stationary_group)
    #daily_results.append(res)
    #print ('prediction time: %ds' % (time.time()-st))
    st = time.time()
    if k == 0:
        continue
    ratio, stationary_ratio_group = lr_fit(train_ap_df, train_keycard_df)
    res, err1, err2, mae1, mae2, pear1, pear2 = ssg_evaluate(ratio, truth, test_ap_df, model, stationary_global, stationary_ratio_group) 
    print ('evaluation time: %ds' % (time.time()-st))
    print ('metric: ',  err1, err2, mae1, mae2, pear1, pear2)
    print ('================================')
    
    metadata_versions.append((deepcopy(metadata[0]), deepcopy(metadata[1]), ratio))
    test_data.append(test_ap_df)
    stationary_ssg_groups.append(deepcopy(stationary_global))
    stationary_ratio_groups.append(deepcopy(stationary_ratio_group))
print ('total running time: %ds' % (time.time()-start))

========iteration 0=========
before preprocess 4889788
after preprocess 4780496
stationary group size: 92
graph running time: 182s
========iteration 1=========
before preprocess 6517265
after preprocess 5082832
stationary group size: 128
graph running time: 100s


/Users/kanghuang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kanghuang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


before preprocess 3751720
after preprocess 3751720


/Users/kanghuang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  # This is added back by InteractiveShellApp.init_path()


evaluation time: 69s
metric:  0.2884447586662175 0.2521821271071432 28.361702127659573 24.634042553191488 0.8903468860427809 0.8872975321495068
========iteration 2=========
before preprocess 19662538
after preprocess 6169012
stationary group size: 142
graph running time: 208s
before preprocess 5629023
after preprocess 5629023
evaluation time: 68s
metric:  0.2684042320028151 0.22063755158038692 25.94333333333333 19.656666666666666 0.929633752011041 0.9363085836592946
========iteration 3=========
before preprocess 9971054
after preprocess 4332157
stationary group size: 153
graph running time: 162s
before preprocess 3864493
after preprocess 3864493
evaluation time: 34s
metric:  0.25038922383608986 0.2361977656001056 28.271966527196653 27.209205020920503 0.9155679287639436 0.9235128683860651
========iteration 4=========
before preprocess 3347030
after preprocess 3278246
stationary group size: 159
graph running time: 113s
before preprocess 3057728
after preprocess 3057728
evaluation time: 4

In [13]:
common_area_truth = pd.read_csv('japan/japan_common_area_manual_counts.csv')
common_area_ap = {37: ['54:ec:2f:29:e1:e0', '54:ec:2f:2a:0d:10', '54:ec:2f:69:cf:ec', '54:ec:2f:69:f3:bc', '54:ec:2f:6a:09:ac', '54:ec:2f:29:af:b0', '54:ec:2f:2a:0c:40'],
                 38: ['54:ec:2f:29:cf:90', '54:ec:2f:29:d6:c0', '54:ec:2f:29:f5:b0', '54:ec:2f:2a:0c:00', '54:ec:2f:29:cf:c0', '54:ec:2f:29:f5:e0', '54:ec:2f:29:f4:20'],
                 39: ['54:ec:2f:29:f3:00', '54:ec:2f:29:f4:b0', '54:ec:2f:29:f6:80', '54:ec:2f:29:f5:ad', '54:ec:2f:29:fc:10', '54:ec:2f:2a:01:40'],
                 #'40': ['1c:3a:60:4c:5e:80', '1c:3a:60:4c:01:00', '1c:3a:60:4c:01:00', '54:ec:2f:69:e2:60'],
                 }
for floor in common_area_ap:
    uppercase = []
    for mac in common_area_ap[floor]:
        uppercase.append(mac.upper())
    common_area_ap[floor] = uppercase

In [14]:
cache = {}

In [86]:
def simulation(k, thresh, common_area_ap=None):
    metadata = metadata_versions[k]
    test_ap_df = test_data[k]
    stationary_ssg = stationary_ssg_groups[k]
    stationary_ratio = stationary_ratio_groups[k]
    model = cooccurrence_transform(metadata[:2], thresh)
    if not common_area_ap is None:
        truth_ = truth
    else:
        truth_ = common_area_truth
    return ssg_evaluate(metadata[2], truth_, test_ap_df, model, stationary_ssg, stationary_ratio, common_area_ap)

In [16]:
for k in tqdm(range(len(metadata_versions)), desc='rounds'):
    for thresh in tqdm(np.arange(0.5, 0.8, 0.01), desc='thresh'):
        if not (k, thresh) in cache:
            cache[(k, thresh)] = simulation(k, thresh)
# res, err1, err2, mae1, mae2, pear1, pear2 = cache[(k, thresh)]


thresh:  19%|█▉        | 6/31 [04:40<19:28, 46.75s/it]


thresh:  45%|████▌     | 14/31 [11:24<13:40, 48.26s/it]


rounds: 100%|██████████| 10/10 [3:41:07<00:00, 1326.78s/it]


In [87]:
res, err1, err2, mae1, mae2, pear1, pear2 = simulation(0, 0.7, common_area_ap)

In [89]:
err1, err2, mae1, mae2, pear1, pear2

(0.28130264812258565,
 0.49165013455619816,
 19.44,
 52.36,
 0.4160858598766494,
 0.3745000712544165)

In [85]:
columns = [ 'error1', 'error2', 'MAE1', 'MAE2', 'pearson1', 'pearson2', 'rounds', 'thresh']
data = []
for k in cache:
    metric = list(cache[k])
    data.append(metric[1:] + list(k))
metric_df = pd.DataFrame(data=data, columns=columns)
metric_df = metric_df.groupby(['rounds']).apply(lambda rows: rows.sort_values('error1').head(1)).reset_index(drop=True)
#metric_df.groupby(['rounds', 'thresh']).apply(lambda rows: rows[rows['err1'] == rows['error1'].min()])
# up = metric_df[(metric_df['thresh'] == 0.54) & (metric_df['rounds'] <= 5)]
# down = metric_df[(metric_df['thresh'] == 0.72) & (metric_df['rounds'] > 5)]
# pd.concat([up, down], axis=0).reset_index(drop=True)

metric_df


,error1,error2,MAE1,MAE2,pearson1,pearson2,rounds,thresh
0,0.245305,0.252182,22.787234,24.634043,0.893370,0.887298,0,0.54
1,0.190968,0.220638,16.803333,19.656667,0.934270,0.936309,1,0.53
2,0.176262,0.236198,19.008368,27.209205,0.918507,0.923513,2,0.53
3,0.208911,0.275857,24.474286,32.057143,0.898277,0.903590,3,0.55
4,0.197699,0.187196,19.944444,19.022222,0.931211,0.942359,4,0.55
5,0.157523,0.188820,16.217877,19.078212,0.951856,0.954051,5,0.59
6,0.189718,0.183328,18.239583,19.489583,0.873427,0.896040,6,0.76
7,0.189202,0.266478,18.720207,28.207254,0.917724,0.917517,7,0.75
8,0.123146,0.140305,13.012579,14.402516,0.950494,0.952319,8,0.73
9,0.268422,0.275529,21.600000,22.700000,0.924130,0.937187,9,0.72


### Keycard to device graph


In [ ]:
timespan = 30
def session(day_keycard_df, day_ap_df): 
    graph = {}
    for _, row in day_keycard_df.iterrows():
        uid = row['uuid']
        if not uid in graph:
            graph[uid] = defaultdict(int)
        timestamp_1st = row['localTimestamp']-timedelta(minutes=5)
        device_1st_df = day_ap_df[day_ap_df['localTimestamp'] < timestamp_1st]
        MAX_TIME = day_ap_df.iloc[0]['date'].replace(hour=23, minute=59, second=59)
        timestamp_2nd = min(MAX_TIME, timestamp_1st + timedelta(minutes=timespan))
        cond = (day_ap_df['localTimestamp'] > timestamp_1st) & (day_ap_df['localTimestamp'] < timestamp_2nd)
        device_2nd_df = day_ap_df[cond]
        bag1 = device_1st_df.uuid.unique()
        bag2 = device_2nd_df.uuid.unique() 
    #     if first:
    #         for i in bag1:
    #             if i in graph[uuid]:
    #                 del graph[uuid][i]
        for i in bag2:
            if (i in graph) or (not i in bag1):
                graph[uid][i] += 1
    return graph

    
def gen_keycard_device_graph(good_ap_df, valid_keycard_df):
    pool = Pool(8)
    valid_keycard_df = valid_keycard_df.sort_values('localTimestamp').reset_index(drop=True)
    res = []
    for key, day_keycard_df in valid_keycard_df.groupby(['date', 'floor_number']):
        date, fl = key
        day_ap_df = good_ap_df[(good_ap_df['date'] == date) & (good_ap_df['floor_number'] == int(fl))]
        print (key, len(day_keycard_df), len(day_ap_df))
        if len(day_ap_df) > 0:
            res.append(pool.apply_async(func=session, args=(day_keycard_df, day_ap_df)))
    print ('wait results ... ')
    pool.close()
    pool.join()
    print ('finished !')
    return res

def gen_prob(kd_graph):
    kv, dv = {}, {}
    for kid in kd_graph:
        if not kid in kv:
            kv[kid] = len(kv)
        for did in kd_graph[kid]:
            if not did in dv:
                dv[did] = len(dv)
    cocur = np.zeros((len(kv), len(dv)))
    kcur = np.zeros((len(kv), 1))
    dcur = np.zeros((1, len(dv)))
    for kid in kd_graph:
        for did in kd_graph[kid]:
            #kcur[kv[kid], 0] += kd_graph[kid][did]
            cocur[kv[kid], dv[did]] = kd_graph[kid][did]
            #dcur[0, dv[did]] += kd_graph[kid][did]
    user_probs = cocur / (np.sum(cocur, axis=0, keepdims=True)+1e-3)
    device_probs = cocur / (np.sum(cocur, axis=1, keepdims=True)+1e-3)
    return user_probs, device_probs, kv, dv

def soft_count(df, model, count_thresh, stationary_group):
    probs, kv, dv = model
    dids = df.uuid.unique()  
    candis = np.array([int(dv[i]) for i in dids if i in dv])
    #new_devices = [i for i in dids if (not i in dv) and (not i in stationary_group)]
#     print (len(new_devices))
    if len(candis) == 0:
        return 0
    # remove noise
    candis_probs = probs[:, candis]
#     print (len(np.where((candis_probs < 0.1) & (candis_probs > 0))[0]))
#     candis_probs[candis_probs < 0.05] = 0
    user = np.argmax(candis_probs, axis=1)
    user_prob = np.max(candis_probs, axis=1)
    valid_user = user[user_prob > 0]
#     candis_probs = candis_probs / (np.sum(candis_probs, axis=0, keepdims=True)+1e-3)
#     count = np.sum(candis_probs, axis=1)
#     count[count > count_thresh] = 1
#     count = np.sum(count)
    return len(np.unique(user))

def filter_low_rssi(valid_ap_df):
    rssi = -80
    rssi_cond = (valid_ap_df['rssi'] < 0) & (valid_ap_df['rssi'] > rssi)
    return valid_ap_df[rssi_cond]

def filter_weekend(df):
    df['date']  = pd.to_datetime(df['date'])
    df['dow'] = df.date.apply(lambda x: x.weekday())
    return df[df.dow < 5]


def preprocess(ap_df, keycard_df=None):
    key_columns = ['localTimestamp', 'floor_number', 'uuid', 'apMac']
    print ('before preprocess %d' % len(ap_df))
    ap_df = filter_low_rssi(ap_df)
    
    ap_df = ap_df[~ap_df.duplicated(key_columns)]
    stationary_group = gen_stationary_group(ap_df, keycard_df)
    ap_df = ap_df[~ap_df.uuid.isin(stationary_group)]
    print ('after preprocess %d' % len(ap_df))
    return ap_df, stationary_group

def simulation(valid_keycard_df, valid_ap_df, count_thresh):
    #overlap, vocab = cooccurrence_model
    good_ap_df = preprocess(valid_ap_df,valid_keycard_df)
    kd_graph = gen_keycard_device_graph(good_ap_df, valid_keycard_df)
    probs, kv, dv = gen_prob(kd_graph)
    #uf = gen_device_groups(overlap, overlap_thresh)
    metadata = {
        'prob': probs,
        #'keycard': valid_keycard_df,
        'kv': kv,
        'dv': dv,
        #'uf': None,
        'thresh': count_thresh,
    }
    
    estimate = good_ap_df.groupby('date').apply(lambda df: soft_count(df, metadata))
    truth = valid_keycard_df.groupby('date').apply(lambda rows: rows.uuid.nunique())
#     print (good_ap_df.groupby('date').apply(lambda rows: len(rows.uuid.unique())))
#     print (estimate)
#     print (truth)
    #return -abs(np.sum((estimate-truth) / truth) / len(truth) * 100)
    return good_ap_df, estimate, truth

In [ ]:
def collector(results):
    graph = {}
    for _res in results:
        res = _res.get()
        merge_graph(graph, res)
    return graph

def merge_graph(graph_large, graph_small):
    for uid in graph_small:
        if not uid in graph_large:
            graph_large[uid] = defaultdict(int)
        for did in graph_small[uid]:
            graph_large[uid][did] += graph_small[uid][did]
    return graph_large

def list_nodes(graph):
    dv = set()
    for kid in graph:
        for did in graph[kid]:
            dv.add(did)
    return dv

def delete_edges(graph, keycard_df, ap_df):
    dates = keycard_df.date.unique()
#     dv = list_nodes(graph)
    for date in dates:
        day_keycard_df = keycard_df[keycard_df.date == date]
        day_ap_df = ap_df[ap_df.date == date]
        kids = day_keycard_df.uuid.unique()
        dids = day_ap_df.uuid.unique()
        for uid in kids:
            devices = list(graph[uid])
            for did in devices:
                if not did in dids:
                    del graph[uid][did]
    return graph

# def delete_edges(graph, edges):
#     for uid in graph:
#         devices = list(graph[uid])
#         for did in devices:
#             if did in edges[uid]:
#                 del graph[uid][did]
#     return graph

def delete_nodes(graph, nodes):
    for uid in graph:
        devices = list(graph[uid])
        for did in devices:
            if did in nodes:
                del graph[uid][did]
    return graph 

def build_graph(train_keycard_df, good_ap_df):
    results = gen_keycard_device_graph(good_ap_df, train_keycard_df)
    kd_graph = collector(results)
    return kd_graph

def clean_graph(graph, nodes, keycard_df, ap_df):
    print ('before clean: %d' % sum([len(graph[i]) for i in graph]))
    graph = delete_nodes(graph, nodes)
    print ('after delete nodes: %d' % sum([len(graph[i]) for i in graph]))
    graph = delete_edges(graph, keycard_df, ap_df)
    print ('after delete edges: %d' % sum([len(graph[i]) for i in graph]))
    return graph



In [ ]:
def create_index():
    date_start = datetime.datetime(2020, 1, 6)
    date_end = datetime.datetime(2020, 3, 24)
    date_groups = []
    cur_date = date_start
    while cur_date <= date_end:
        date_groups.append((cur_date, min(date_end, cur_date+timedelta(days=6))))
        cur_date += timedelta(days=7)
    return date_groups

def prepare_data(k, date_groups):
    train_ap_df = valid_ap_df[(valid_ap_df.date >= date_groups[k][0]) & (valid_ap_df.date <= date_groups[k][1])]
    test_ap_df = valid_ap_df[(valid_ap_df.date >= date_groups[k+1][0]) & (valid_ap_df.date <= date_groups[k+1][1])]
    train_keycard_df = valid_keycard_df[(valid_keycard_df.date >= date_groups[k][0]) & (valid_keycard_df.date <= date_groups[k][1])]
    test_keycard_df = valid_keycard_df[(valid_keycard_df.date >= date_groups[k+1][0]) & (valid_keycard_df.date <= date_groups[k+1][1])]
    return train_ap_df, test_ap_df, train_keycard_df, test_keycard_df

def update(graph_large, nodes_large, train_keycard_df, train_ap_df):
    st = time.time()
    train_ap_df = filter_weekend(train_ap_df)
    train_keycard_df = filter_weekend(train_keycard_df)
    good_ap_df, nodes_small = preprocess(train_ap_df, train_keycard_df)
    good_ap_df = good_ap_df[~good_ap_df.uuid.isin(nodes_large)]
    graph_small = build_graph(train_keycard_df, good_ap_df)
    graph_small = clean_graph(graph_small, nodes_small, train_keycard_df, good_ap_df)
    nodes = nodes_large.union(nodes_small)
    print ('stationary nodes: %d' % len(nodes))
    graph = merge_graph(graph_large, graph_small)
    print ('running time %d'% (time.time()-st))
    return graph, nodes

In [ ]:
ap_df = train_ap_df[~train_ap_df.duplicated(['uuid', 'floor_number', 'apMac', 'localTimestamp'])]
occupancy = ap_df.groupby('uuid').apply(lambda df: df.groupby('date').apply(lambda rows: len(rows) / (24*60*2.0)).max())

In [ ]:
date_groups = create_index()
graph = {}
block_nodes = set()
for k in range(2):
    train_ap_df, test_ap_df, train_keycard_df, test_keycard_df = prepare_data(k, date_groups)
    graph, block_nodes = update(graph, block_nodes, train_keycard_df, train_ap_df)


In [ ]:
counts = sorted([len(graph[i]) for i in graph])
c = 0
for i in counts:
    if i == 0:
        c += 1
print (c, len(counts) - c, max(counts))

In [ ]:
def predict(test_keycard_df, test_ap_df, graph, stationary_group): 
    good_ap_df = test_ap_df[~test_ap_df.uuid.isin(stationary_group)]
    probs, _, kv, dv = gen_prob(graph)
    model_v1 = probs, kv, dv
    estimate = good_ap_df.groupby('date').apply(lambda df: soft_count(df, model_v1, .5, stationary_group))
    keycard = test_keycard_df.groupby('date').apply(lambda rows: rows.uuid.nunique())
    delta = test_keycard_df.groupby('date').apply(lambda rows: rows[~rows.uuid.isin(kv)].uuid.nunique())
    device = good_ap_df.groupby('date').apply(lambda rows: rows.uuid.nunique())
  
    dashboard = pd.concat([estimate, keycard, device, delta], axis=1).reset_index(drop=False)
    dashboard = dashboard[~dashboard[0].isna()]
    
    #truth = pd.read_csv('japan/japan_manual_counts.csv')
    return dashboard

predict(test_keycard_df, test_ap_df, graph, block_nodes)


In [ ]:
truth = pd.read_csv('japan/japan_manual_counts.csv')
truth = truth[truth['building_id'] == 22]


In [ ]:
def evaluate(truth, test_ap_df, graph, stationary_group):  
    probs, _, kv, dv = gen_prob(graph)
    model_v1 = probs, kv, dv
    good_ap_df = test_ap_df[~test_ap_df.uuid.isin(stationary_group)]
    truth['time_local'] = pd.to_datetime(truth['time_local'])
    truth = truth[(truth['time_local'] >= test_ap_df.date.min()) & (truth['time_local'] <= test_ap_df.date.max())].reset_index(drop=True)
    
    for i, row in truth.iterrows():
        timestamp = row['time_local']
        cond = (good_ap_df['localTimestamp'] > (row['time_local'] - timedelta(minutes=5)))&(good_ap_df['localTimestamp'] < (row['time_local'] + timedelta(minutes=0)))
        cond &=  (good_ap_df['floor_number'] == row['floor'])
        ap_df = good_ap_df[cond]
        truth.loc[i, 'device'] = ap_df.uuid.nunique()
        truth.loc[i, 'estimate'] = soft_count(ap_df, model_v1, .5, stationary_group)
        #print (ap_df.uuid.nunique(), row['manual_count'], count, timestamp, row['floor'])
    return truth.sort_values(['floor', 'time_local']).reset_index(drop=True)
res = evaluate(truth, test_ap_df, graph, block_nodes)

In [ ]:
from bayes_opt import BayesianOptimization    
from bayes_opt import SequentialDomainReductionTransformer

black_box_function = functools.partial(simulation, valid_keycard_df, valid_ap_df, model)
pbounds = {'count_thresh': (0, 1)}
bounds_transformer = SequentialDomainReductionTransformer()
mutating_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=1,
    random_state=7,
    bounds_transformer=bounds_transformer
)
st = time.time()
mutating_optimizer.maximize(
    init_points=2,
    n_iter=30,
)
print ('running time %d'% (time.time()-st))